In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import convolve
import cv2
import pickle

In [3]:
def init_tumor2(size, radius):
    """
     initializes the field phi with a circle
     :param size: size of matrix
     :param radius: tumor initial radius (created in center)
     :return param phi: order parameter matrix
    """
    cx,cy = int(size/2),int(size/2)
    x, y = np.indices((size, size))
    phi = (np.abs(np.hypot(x-cx, y-cy)) < radius).astype(float)
    return phi

In [4]:
def init_tumor_ellipse(size,a,b,radius):
    cx,cy = int(size/2),int(size/2)
    x, y = np.indices((size, size))
    phi = (np.abs(np.hypot((x-cx)/a, (y-cy)/b)) < radius).astype(float)
    return phi

In [5]:
def init_s2(size):
    s = 2.55 + 0.4 * np.random.rand(size,size)
    return s

In [6]:
def fintegrate_nophi(phi, sigma, S, dt, h, **kwargs):
    """
      try to go even faster
      -> don't create copy here
      -> try to reduce multiplications
      -> cv2 for convolution?
      integrate the order parameter phi and the chemical field sigma one tstep
      with 5x5 stencil (five point stencil - https://en.wikipedia.org/wiki/Five-point_stencil)
    :param phi: order parameter
    :param sigma: nutrient
    :param S: source of nutrient
    :param tstep: total number of timesteps
    :param dt: time increment
    :param kwargs: dictionary of parameters lambda, tau, chi, A, epsilon, delta, gamma
    :return:
    """
    stencil_5 = (1.0 / (12.0 * h * h)) * np.array(
        [[0, 0, -1, 0, 0],
         [0, 0, 16, 0, 0],
         [-1, 16, -60, 16, -1],
         [0, 0, 16, 0, 0],
         [0, 0, -1, 0, 0]])

    phinew = phi + dt * (kwargs['lambda_'] * cv2.filter2D(phi,-1,stencil_5,borderType=cv2.BORDER_REPLICATE) +
                      (64.0/kwargs['tau']) * phi * (1.0 - phi) * (phi - 0.5) +
                      kwargs['chi'] * sigma - kwargs['A'] * phi)

    sigmanew = sigma + dt * (kwargs['epsilon']*kwargs['lambda_'] * cv2.filter2D(sigma,-1,stencil_5,borderType=cv2.BORDER_REPLICATE) +
                              S - kwargs['delta'] * phi - kwargs['gamma'] * sigma)    

    return phinew, sigmanew

In [7]:
def run_partest(phistart, sigmastart, Sstart, param):
    print("running parameter test with parameters", param)
    phiold = np.copy(phistart)
    sigmaold = np.copy(sigmastart)
    S = np.copy(Sstart)
    phiarr = np.zeros((param['days'],size,size)) #take care, size not derived from input!
    sigmaarr = np.zeros((param['days'],size,size))
    
    tstep = param['days']/param['dt']
    
    for iteration in np.arange(tstep):
        phinew, sigmanew = fintegrate_nophi(phiold, sigmaold, S, **param)
        phiold = phinew
        sigmaold = sigmanew

        if iteration%int(1/param['dt']) == 0:
            phiarr[int(iteration*param['dt'])]=phinew
            sigmaarr[int(iteration*param['dt'])]=sigmanew
            print("day",iteration*param['dt'])
            #plt.imshow(phiold,vmin=0,vmax=1)
            #plt.imshow(sigmaold,vmin=0,vmax=3)
            #plt.show()    
    results = {'param':param,'phistart':phistart,'sigmastart':sigmastart,'Sstart':Sstart,'phiarr':phiarr,'sigmaarr':sigmaarr}
    return results

In [12]:
#Avals = list(np.arange(1.3,3.0,0.1))
Avals = [1.5,1.6,1.7]
chivals = [1.5]
#deltavals = list(np.arange(2.2,3.2,0.1))

In [13]:
size = 600
phistart = init_tumor_ellipse(size,1,1.3,50)
sigmastart = np.ones((size,size),dtype=float)
Sstart = init_s2(size)
param = {'dt':0.00025, 'h':5., 'lambda_':432., 'epsilon':40., 
         'A':1.5, 'gamma':2.74, 'tau':3.65, 'chi':1.5, 'delta':2.75, 'days':365}

for A in Avals:
    for chi in chivals:
        param['A'] = A
        param['chi'] = chi
        res = run_partest(phistart, sigmastart, Sstart, param)
        
        filename = '/home/alunos/Desktop/parvariation_A_'+str(A)+'_chi_'+str(chi)+'.pickle'
        with open(filename, 'wb') as handle:
            pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

running parameter test with parameters {'days': 365, 'lambda_': 432.0, 'gamma': 2.74, 'chi': 1.5, 'delta': 2.75, 'tau': 3.65, 'dt': 0.00025, 'epsilon': 40.0, 'h': 5.0, 'A': 1.5}
day 0.0
day 1.0
day 2.0
day 3.0
day 4.0
day 5.0
day 6.0
day 7.0
day 8.0
day 9.0
day 10.0
day 11.0
day 12.0
day 13.0
day 14.0
day 15.0
day 16.0
day 17.0
day 18.0
day 19.0
day 20.0
day 21.0
day 22.0
day 23.0
day 24.0
day 25.0
day 26.0
day 27.0
day 28.0
day 29.0
day 30.0
day 31.0
day 32.0
day 33.0
day 34.0
day 35.0
day 36.0
day 37.0
day 38.0
day 39.0
day 40.0
day 41.0
day 42.0
day 43.0
day 44.0
day 45.0
day 46.0
day 47.0
day 48.0
day 49.0
day 50.0
day 51.0
day 52.0
day 53.0
day 54.0
day 55.0
day 56.0
day 57.0
day 58.0
day 59.0
day 60.0
day 61.0
day 62.0
day 63.0
day 64.0
day 65.0
day 66.0
day 67.0
day 68.0
day 69.0
day 70.0
day 71.0
day 72.0
day 73.0
day 74.0
day 75.0
day 76.0
day 77.0
day 78.0
day 79.0
day 80.0
day 81.0
day 82.0
day 83.0
day 84.0
day 85.0
day 86.0
day 87.0
day 88.0
day 89.0
day 90.0
day 91.0
day 

day 66.0
day 67.0
day 68.0
day 69.0
day 70.0
day 71.0
day 72.0
day 73.0
day 74.0
day 75.0
day 76.0
day 77.0
day 78.0
day 79.0
day 80.0
day 81.0
day 82.0
day 83.0
day 84.0
day 85.0
day 86.0
day 87.0
day 88.0
day 89.0
day 90.0
day 91.0
day 92.0
day 93.0
day 94.0
day 95.0
day 96.0
day 97.0
day 98.0
day 99.0
day 100.0
day 101.0
day 102.0
day 103.0
day 104.0
day 105.0
day 106.0
day 107.0
day 108.0
day 109.0
day 110.0
day 111.0
day 112.0
day 113.0
day 114.0
day 115.0
day 116.0
day 117.0
day 118.0
day 119.0
day 120.0
day 121.0
day 122.0
day 123.0
day 124.0
day 125.0
day 126.0
day 127.0
day 128.0
day 129.0
day 130.0
day 131.0
day 132.0
day 133.0
day 134.0
day 135.0
day 136.0
day 137.0
day 138.0
day 139.0
day 140.0
day 141.0
day 142.0
day 143.0
day 144.0
day 145.0
day 146.0
day 147.0
day 148.0
day 149.0
day 150.0
day 151.0
day 152.0
day 153.0
day 154.0
day 155.0
day 156.0
day 157.0
day 158.0
day 159.0
day 160.0
day 161.0
day 162.0
day 163.0
day 164.0
day 165.0
day 166.0
day 167.0
day 168.0
day 